In [1]:
import pandas as pd
import numpy as np
import pyter

In [2]:
# Fake_ter is only for demostration and testing when pyter is not available.
# The function compares the exact difference between two strings.
from difflib import SequenceMatcher 

def fake_ter(hyp, ref):
    seq = SequenceMatcher(None,hyp,ref)
    return seq.ratio()

In [3]:
path = "models\sy_transformer_wmt17_ende_groundhog_3\hyps_50.csv"
num_rows = 100
r_data = pd.read_csv(path, nrows=num_rows, index_col=[0])
r_data = r_data.drop(['TER scores','Z_scores'], axis = 1)
data = r_data[50:100]
#print(data.head())
data = data.reset_index(drop=True)
data.style.set_properties(**{'text-align': 'left'})
pd.set_option('display.max_colwidth', 120)
data.head()


,Predictions,Scores
0,Der neue indische Premierminister Narendra Modi trifft seinen japanischen Amtskollegen Shinzo Abe in Tokio anlässlic...,-1.983042
1,Der neue indische Premierminister Narendra Modi trifft sich mit seinem japanischen Amtskollegen Shinzo Abe in Tokio ...,-1.992233
2,Der neue indische Premierminister Narendra Modi trifft sich mit seinem japanischen Amtskollegen Shinzo Abe in Tokio ...,-1.992458
3,"Indiens neuer Premierminister , Narendra Modi , trifft sich mit seinem japanischen Amtskollegen Shinzo Abe in Tokio ...",-1.993631
4,"Indiens neuer Premierminister , Narendra Modi , trifft sich mit seinem japanischen Amtskollegen Shinzo Abe in Tokio ...",-1.994340


In [4]:
def find_seq_list(data:pd.DataFrame, n:int, alpha=1.0, beta=1.0):
    memo_ter = [[] for _ in range(n-1)]
    selected_list = []
    remaining_list = [bool(1) for _ in range(len(data.index))]
  

    def cal_seq(data:pd.DataFrame, selected_list:list, remaining_list:list, n: int, memo_ter: list, alpha:float, beta:float):
    
        assert n > 0, "Number of selected predictions has to be a positive integer."
    
        if n == 1:
            selected_idx = np.argmax(data["Scores"].to_numpy())
            selected_list.append(selected_idx)
            remaining_list[selected_idx] = False
            return selected_list, remaining_list, memo_ter
    
        if n > 1:
            selected_list, remaining_list, memo_ter = cal_seq(data, selected_list, remaining_list, n-1, memo_ter,alpha, beta)

            print(n-2)
            ter_list = [[] for _ in range(len(data.index))]
            ref = data["Predictions"][selected_list[n-2]]
            for iter_i in range(len(data.index)): 
                if remaining_list[iter_i] == 0:  
                    # exclude elements in selected_list as hypotheses to be compared to ref
                    ter_list[iter_i] = 0.0
                else: 
                    ter_list[iter_i] = pyter.ter(data["Predictions"][iter_i], ref)
                    #ter_list[iter_i] = fake_ter(data["Predictions"][iter_i], ref)

            memo_ter[n-2] = ter_list
            print("second")
            # save ter_list for i step to memo_ter

            sum_ter = np.zeros((len(data.index),1))
            z_scores = np.ones(sum_ter.shape)*(-np.inf)
            for j in range(n-1):
                ter = np.array(memo_ter[j],dtype = np.float64).reshape(50,1)               
                sum_ter = sum_ter + ter
            sum_ter = sum_ter/(n-1)
            z_scores[remaining_list] = alpha * np.array(data["Scores"][remaining_list]).reshape(-1,1) + beta * sum_ter[remaining_list]
            selected_idx = np.argmax(z_scores)
            selected_list.append(selected_idx)
            remaining_list[selected_idx] = False
      
            return selected_list, remaining_list, memo_ter
    
    selected_ls, remaining_ls, ter_ls = cal_seq(data, selected_list, remaining_list, n, memo_ter, alpha, beta)
    print("third")
    print(len(ter_ls))
    return selected_ls, remaining_ls, ter_ls

In [5]:
selected_ls, remaining_ls, ter_ls = find_seq_list(data,4)

0
second
1
second
2
second
third
3


In [6]:
print(selected_ls)
print(remaining_ls)
print(len(ter_ls))

[0, 25, 14, 7]
[False, True, True, True, True, True, True, False, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
3


In [ ]:
sub_df = data[["Predictions","Scores"]]
sub_df.iloc[selected_ls]